# GraphSAGE on ogbn-proteins

from https://github.com/snap-stanford/ogb/blob/master/examples/nodeproppred/proteins/gnn.py by Matthias Fey

In [1]:
import argparse

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

In [2]:
class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [4]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x

In [5]:
def train(model, data, train_idx, optimizer):
    model.train()
    criterion = torch.nn.BCEWithLogitsLoss()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    loss = criterion(out, data.y[train_idx].to(torch.float))
    loss.backward()
    optimizer.step()

    return loss.item()

In [6]:
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    y_pred = model(data.x, data.adj_t)

    train_rocauc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['rocauc']
    valid_rocauc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['rocauc']
    test_rocauc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['rocauc']

    return train_rocauc, valid_rocauc, test_rocauc

In [21]:
parser = argparse.ArgumentParser(description='OGBN-Proteins (GNN)')
parser.add_argument('--device', type=int, default=0)
parser.add_argument('--log_steps', type=int, default=1)
parser.add_argument('--use_sage', action='store_true', default=True)
parser.add_argument('--num_layers', type=int, default=3)
parser.add_argument('--hidden_channels', type=int, default=256)
parser.add_argument('--dropout', type=float, default=0.0)
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epochs', type=int, default=1000)
parser.add_argument('--eval_steps', type=int, default=5)
parser.add_argument('--runs', type=int, default=10)
args = parser.parse_args(args=[])
print(args)

device = f'cuda:{args.device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

dataset = PygNodePropPredDataset(
    name='ogbn-proteins', transform=T.ToSparseTensor(attr='edge_attr'))
data = dataset[0]

# Move edge features to node features.
data.x = data.adj_t.mean(dim=1)
data.adj_t.set_value_(None)

split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

model = SAGE(data.num_features, args.hidden_channels, 112,
                 args.num_layers, args.dropout).to(device)

data = data.to(device)

evaluator = Evaluator(name='ogbn-proteins')
logger = Logger(args.runs, args)

Namespace(device=0, dropout=0.0, epochs=1000, eval_steps=5, hidden_channels=256, log_steps=1, lr=0.01, num_layers=3, runs=10, use_sage=True)


SAGE

In [22]:
for run in range(args.runs):
    model.reset_parameters()
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    for epoch in range(1, 1 + args.epochs):
        loss = train(model, data, train_idx, optimizer)

        if epoch % args.eval_steps == 0:
            result = test(model, data, split_idx, evaluator)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_rocauc, valid_rocauc, test_rocauc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_rocauc:.2f}%, '
                      f'Valid: {100 * valid_rocauc:.2f}% '
                      f'Test: {100 * test_rocauc:.2f}%')

    logger.print_statistics(run)
logger.print_statistics()

Run: 01, Epoch: 05, Loss: 0.3567, Train: 65.63%, Valid: 56.85% Test: 49.60%
Run: 01, Epoch: 10, Loss: 0.3476, Train: 66.56%, Valid: 54.98% Test: 47.63%
Run: 01, Epoch: 15, Loss: 0.3427, Train: 67.22%, Valid: 55.62% Test: 48.94%
Run: 01, Epoch: 20, Loss: 0.3369, Train: 67.85%, Valid: 59.54% Test: 51.04%
Run: 01, Epoch: 25, Loss: 0.3329, Train: 69.34%, Valid: 62.89% Test: 54.84%
Run: 01, Epoch: 30, Loss: 0.3266, Train: 70.24%, Valid: 67.04% Test: 60.17%
Run: 01, Epoch: 35, Loss: 0.3241, Train: 71.09%, Valid: 68.87% Test: 63.00%
Run: 01, Epoch: 40, Loss: 0.3196, Train: 72.09%, Valid: 69.84% Test: 64.73%
Run: 01, Epoch: 45, Loss: 0.3161, Train: 73.18%, Valid: 70.86% Test: 66.03%
Run: 01, Epoch: 50, Loss: 0.3122, Train: 74.10%, Valid: 71.39% Test: 66.71%
Run: 01, Epoch: 55, Loss: 0.3083, Train: 74.82%, Valid: 72.24% Test: 67.99%
Run: 01, Epoch: 60, Loss: 0.3054, Train: 75.63%, Valid: 73.17% Test: 68.83%
Run: 01, Epoch: 65, Loss: 0.3017, Train: 76.66%, Valid: 74.37% Test: 69.33%
Run: 01, Epo

Run: 01, Epoch: 540, Loss: 0.2483, Train: 86.32%, Valid: 82.08% Test: 76.09%
Run: 01, Epoch: 545, Loss: 0.2477, Train: 86.37%, Valid: 82.18% Test: 76.30%
Run: 01, Epoch: 550, Loss: 0.2481, Train: 86.40%, Valid: 82.24% Test: 76.37%
Run: 01, Epoch: 555, Loss: 0.2478, Train: 86.42%, Valid: 82.16% Test: 76.10%
Run: 01, Epoch: 560, Loss: 0.2525, Train: 86.39%, Valid: 82.37% Test: 76.83%
Run: 01, Epoch: 565, Loss: 0.2476, Train: 86.43%, Valid: 82.39% Test: 76.85%
Run: 01, Epoch: 570, Loss: 0.2482, Train: 86.47%, Valid: 82.35% Test: 76.58%
Run: 01, Epoch: 575, Loss: 0.2480, Train: 86.51%, Valid: 82.39% Test: 76.62%
Run: 01, Epoch: 580, Loss: 0.2474, Train: 86.54%, Valid: 82.34% Test: 76.39%
Run: 01, Epoch: 585, Loss: 0.2470, Train: 86.57%, Valid: 82.35% Test: 76.44%
Run: 01, Epoch: 590, Loss: 0.2461, Train: 86.60%, Valid: 82.41% Test: 76.53%
Run: 01, Epoch: 595, Loss: 0.2466, Train: 86.63%, Valid: 82.35% Test: 76.26%
Run: 01, Epoch: 600, Loss: 0.2484, Train: 86.62%, Valid: 82.50% Test: 76.73%

Run: 02, Epoch: 70, Loss: 0.2978, Train: 77.77%, Valid: 74.95% Test: 69.84%
Run: 02, Epoch: 75, Loss: 0.2943, Train: 78.55%, Valid: 75.62% Test: 70.34%
Run: 02, Epoch: 80, Loss: 0.2918, Train: 79.08%, Valid: 76.03% Test: 70.64%
Run: 02, Epoch: 85, Loss: 0.2897, Train: 79.50%, Valid: 76.39% Test: 70.86%
Run: 02, Epoch: 90, Loss: 0.2879, Train: 79.86%, Valid: 76.59% Test: 71.03%
Run: 02, Epoch: 95, Loss: 0.2862, Train: 80.21%, Valid: 76.85% Test: 71.33%
Run: 02, Epoch: 100, Loss: 0.2879, Train: 80.48%, Valid: 77.13% Test: 71.58%
Run: 02, Epoch: 105, Loss: 0.2840, Train: 80.70%, Valid: 77.18% Test: 71.34%
Run: 02, Epoch: 110, Loss: 0.2830, Train: 80.92%, Valid: 77.34% Test: 71.64%
Run: 02, Epoch: 115, Loss: 0.2827, Train: 81.11%, Valid: 77.46% Test: 71.82%
Run: 02, Epoch: 120, Loss: 0.2809, Train: 81.33%, Valid: 77.59% Test: 71.84%
Run: 02, Epoch: 125, Loss: 0.2795, Train: 81.54%, Valid: 77.75% Test: 72.01%
Run: 02, Epoch: 130, Loss: 0.2793, Train: 81.72%, Valid: 77.93% Test: 72.32%
Run: 

Run: 02, Epoch: 605, Loss: 0.2458, Train: 86.64%, Valid: 82.18% Test: 75.80%
Run: 02, Epoch: 610, Loss: 0.2467, Train: 86.65%, Valid: 82.09% Test: 75.49%
Run: 02, Epoch: 615, Loss: 0.2584, Train: 86.58%, Valid: 82.27% Test: 76.18%
Run: 02, Epoch: 620, Loss: 0.2504, Train: 86.65%, Valid: 82.11% Test: 75.50%
Run: 02, Epoch: 625, Loss: 0.2459, Train: 86.67%, Valid: 82.08% Test: 75.53%
Run: 02, Epoch: 630, Loss: 0.2478, Train: 86.69%, Valid: 82.15% Test: 75.69%
Run: 02, Epoch: 635, Loss: 0.2456, Train: 86.72%, Valid: 82.27% Test: 75.93%
Run: 02, Epoch: 640, Loss: 0.2457, Train: 86.75%, Valid: 82.22% Test: 75.79%
Run: 02, Epoch: 645, Loss: 0.2449, Train: 86.77%, Valid: 82.23% Test: 75.76%
Run: 02, Epoch: 650, Loss: 0.2448, Train: 86.80%, Valid: 82.27% Test: 75.85%
Run: 02, Epoch: 655, Loss: 0.2445, Train: 86.82%, Valid: 82.32% Test: 75.91%
Run: 02, Epoch: 660, Loss: 0.2442, Train: 86.85%, Valid: 82.31% Test: 75.87%
Run: 02, Epoch: 665, Loss: 0.2441, Train: 86.87%, Valid: 82.39% Test: 76.05%

Run: 03, Epoch: 135, Loss: 0.2796, Train: 81.91%, Valid: 77.89% Test: 71.87%
Run: 03, Epoch: 140, Loss: 0.2777, Train: 82.07%, Valid: 78.03% Test: 72.11%
Run: 03, Epoch: 145, Loss: 0.2765, Train: 82.23%, Valid: 78.13% Test: 72.21%
Run: 03, Epoch: 150, Loss: 0.2755, Train: 82.38%, Valid: 78.24% Test: 72.32%
Run: 03, Epoch: 155, Loss: 0.2758, Train: 82.51%, Valid: 78.45% Test: 72.69%
Run: 03, Epoch: 160, Loss: 0.2747, Train: 82.63%, Valid: 78.55% Test: 72.79%
Run: 03, Epoch: 165, Loss: 0.2726, Train: 82.76%, Valid: 78.67% Test: 72.98%
Run: 03, Epoch: 170, Loss: 0.2741, Train: 82.88%, Valid: 78.72% Test: 72.94%
Run: 03, Epoch: 175, Loss: 0.2714, Train: 82.99%, Valid: 78.81% Test: 73.03%
Run: 03, Epoch: 180, Loss: 0.2726, Train: 83.07%, Valid: 78.97% Test: 73.41%
Run: 03, Epoch: 185, Loss: 0.2707, Train: 83.18%, Valid: 79.04% Test: 73.46%
Run: 03, Epoch: 190, Loss: 0.2702, Train: 83.28%, Valid: 79.09% Test: 73.46%
Run: 03, Epoch: 195, Loss: 0.2691, Train: 83.37%, Valid: 79.13% Test: 73.47%

...

Run: 08, Epoch: 460, Loss: 0.2526, Train: 86.33%, Valid: 81.81% Test: 76.61%
Run: 08, Epoch: 465, Loss: 0.2495, Train: 86.37%, Valid: 81.82% Test: 76.53%
Run: 08, Epoch: 470, Loss: 0.2483, Train: 86.42%, Valid: 81.87% Test: 76.65%
Run: 08, Epoch: 475, Loss: 0.2472, Train: 86.46%, Valid: 81.88% Test: 76.59%
Run: 08, Epoch: 480, Loss: 0.2473, Train: 86.49%, Valid: 81.84% Test: 76.45%
Run: 08, Epoch: 485, Loss: 0.2471, Train: 86.52%, Valid: 81.81% Test: 76.32%
Run: 08, Epoch: 490, Loss: 0.2500, Train: 86.52%, Valid: 82.00% Test: 76.75%
Run: 08, Epoch: 495, Loss: 0.2462, Train: 86.57%, Valid: 81.96% Test: 76.63%
Run: 08, Epoch: 500, Loss: 0.2462, Train: 86.60%, Valid: 81.95% Test: 76.57%
Run: 08, Epoch: 505, Loss: 0.2467, Train: 86.63%, Valid: 81.92% Test: 76.44%
Run: 08, Epoch: 510, Loss: 0.2462, Train: 86.66%, Valid: 81.93% Test: 76.40%
Run: 08, Epoch: 515, Loss: 0.2461, Train: 86.70%, Valid: 82.08% Test: 76.70%
Run: 08, Epoch: 520, Loss: 0.2473, Train: 86.72%, Valid: 81.97% Test: 76.37%

Run: 08, Epoch: 995, Loss: 0.2324, Train: 88.30%, Valid: 83.43% Test: 77.27%
Run: 08, Epoch: 1000, Loss: 0.2320, Train: 88.31%, Valid: 83.40% Test: 77.19%
Run 08:
Highest Train: 88.31
Highest Valid: 83.43
  Final Train: 88.07
   Final Test: 77.48
Run: 09, Epoch: 05, Loss: 0.3561, Train: 61.27%, Valid: 59.35% Test: 57.64%
Run: 09, Epoch: 10, Loss: 0.3532, Train: 66.83%, Valid: 63.50% Test: 58.80%
Run: 09, Epoch: 15, Loss: 0.3463, Train: 67.74%, Valid: 59.75% Test: 53.59%
Run: 09, Epoch: 20, Loss: 0.3387, Train: 68.09%, Valid: 60.65% Test: 52.93%
Run: 09, Epoch: 25, Loss: 0.3335, Train: 68.89%, Valid: 61.82% Test: 54.93%
Run: 09, Epoch: 30, Loss: 0.3265, Train: 70.24%, Valid: 66.10% Test: 59.91%
Run: 09, Epoch: 35, Loss: 0.3242, Train: 71.30%, Valid: 69.12% Test: 64.17%
Run: 09, Epoch: 40, Loss: 0.3198, Train: 72.19%, Valid: 69.64% Test: 65.56%
Run: 09, Epoch: 45, Loss: 0.3160, Train: 73.09%, Valid: 70.29% Test: 66.56%
Run: 09, Epoch: 50, Loss: 0.3115, Train: 74.12%, Valid: 71.52% Test: 

Run: 09, Epoch: 525, Loss: 0.2509, Train: 86.31%, Valid: 81.87% Test: 76.34%
Run: 09, Epoch: 530, Loss: 0.2498, Train: 86.33%, Valid: 82.02% Test: 76.79%
Run: 09, Epoch: 535, Loss: 0.2483, Train: 86.37%, Valid: 81.93% Test: 76.46%
Run: 09, Epoch: 540, Loss: 0.2492, Train: 86.40%, Valid: 81.96% Test: 76.47%
Run: 09, Epoch: 545, Loss: 0.2483, Train: 86.44%, Valid: 82.04% Test: 76.70%
Run: 09, Epoch: 550, Loss: 0.2474, Train: 86.46%, Valid: 82.07% Test: 76.79%
Run: 09, Epoch: 555, Loss: 0.2469, Train: 86.49%, Valid: 82.13% Test: 76.91%
Run: 09, Epoch: 560, Loss: 0.2469, Train: 86.52%, Valid: 82.14% Test: 76.91%
Run: 09, Epoch: 565, Loss: 0.2469, Train: 86.54%, Valid: 82.09% Test: 76.75%
Run: 09, Epoch: 570, Loss: 0.2467, Train: 86.57%, Valid: 82.22% Test: 77.04%
Run: 09, Epoch: 575, Loss: 0.2465, Train: 86.59%, Valid: 82.17% Test: 76.85%
Run: 09, Epoch: 580, Loss: 0.2512, Train: 86.57%, Valid: 82.33% Test: 77.30%
Run: 09, Epoch: 585, Loss: 0.2463, Train: 86.58%, Valid: 82.37% Test: 77.25%

Run: 10, Epoch: 55, Loss: 0.3059, Train: 75.74%, Valid: 73.45% Test: 68.76%
Run: 10, Epoch: 60, Loss: 0.3012, Train: 76.88%, Valid: 74.53% Test: 69.29%
Run: 10, Epoch: 65, Loss: 0.2967, Train: 77.88%, Valid: 75.56% Test: 70.10%
Run: 10, Epoch: 70, Loss: 0.2936, Train: 78.60%, Valid: 76.26% Test: 70.60%
Run: 10, Epoch: 75, Loss: 0.2913, Train: 79.13%, Valid: 76.53% Test: 70.59%
Run: 10, Epoch: 80, Loss: 0.2895, Train: 79.51%, Valid: 76.71% Test: 70.92%
Run: 10, Epoch: 85, Loss: 0.2882, Train: 79.82%, Valid: 76.86% Test: 71.20%
Run: 10, Epoch: 90, Loss: 0.2874, Train: 80.14%, Valid: 77.05% Test: 71.42%
Run: 10, Epoch: 95, Loss: 0.2849, Train: 80.43%, Valid: 77.16% Test: 71.53%
Run: 10, Epoch: 100, Loss: 0.2836, Train: 80.70%, Valid: 77.35% Test: 71.75%
Run: 10, Epoch: 105, Loss: 0.2826, Train: 80.95%, Valid: 77.52% Test: 72.02%
Run: 10, Epoch: 110, Loss: 0.2825, Train: 81.21%, Valid: 77.75% Test: 72.20%
Run: 10, Epoch: 115, Loss: 0.2800, Train: 81.43%, Valid: 77.88% Test: 72.45%
Run: 10,

Run: 10, Epoch: 590, Loss: 0.2453, Train: 86.94%, Valid: 82.29% Test: 76.20%
Run: 10, Epoch: 595, Loss: 0.2448, Train: 86.96%, Valid: 82.44% Test: 76.52%
Run: 10, Epoch: 600, Loss: 0.2431, Train: 87.00%, Valid: 82.39% Test: 76.35%
Run: 10, Epoch: 605, Loss: 0.2428, Train: 87.02%, Valid: 82.39% Test: 76.31%
Run: 10, Epoch: 610, Loss: 0.2512, Train: 86.92%, Valid: 82.54% Test: 76.70%
Run: 10, Epoch: 615, Loss: 0.2508, Train: 86.77%, Valid: 82.52% Test: 77.11%
Run: 10, Epoch: 620, Loss: 0.2459, Train: 86.86%, Valid: 81.98% Test: 75.45%
Run: 10, Epoch: 625, Loss: 0.2492, Train: 86.89%, Valid: 82.23% Test: 76.04%
Run: 10, Epoch: 630, Loss: 0.2474, Train: 86.96%, Valid: 82.27% Test: 76.07%
Run: 10, Epoch: 635, Loss: 0.2436, Train: 87.00%, Valid: 82.26% Test: 76.05%
Run: 10, Epoch: 640, Loss: 0.2428, Train: 87.04%, Valid: 82.37% Test: 76.28%
Run: 10, Epoch: 645, Loss: 0.2426, Train: 87.09%, Valid: 82.35% Test: 76.19%
Run: 10, Epoch: 650, Loss: 0.2420, Train: 87.13%, Valid: 82.51% Test: 76.52%